### 0.Import package

In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import requests, json
from pandas import json_normalize
import json
from datetime import datetime

### 1. Get Data

In [2]:
from fdrsapi import api_function,baseline, api_function_imputed

In [3]:
f = open("public_api_key.txt", "r")
api_key=f.readline()
years=["2016","2017","2018","2019","2020"]
kpi_code=["KPI_DonBlood_Tot","KPI_TrainFA_Tot","KPI_ReachDRER_D_Tot","KPI_ReachDRER_I","KPI_ReachDRR_D_Tot","KPI_ReachDRR_I","KPI_ReachLTSPD_D_Tot","KPI_ReachLTSPD_I","KPI_ReachS_D_Tot","KPI_ReachS_I","KPI_ReachL_D_Tot","KPI_ReachL_I","KPI_ReachM_D_Tot","KPI_ReachM_I","KPI_ReachCTP_D_Tot","KPI_ReachCTP_I","KPI_ReachSI_D_Tot","KPI_ReachSI_I","KPI_ReachH_D_Tot","KPI_ReachH_I","KPI_ReachWASH_I","KPI_ReachWASH_D_Tot","KPI_ReachDRER_CPD","KPI_ReachLTSPD_CPD","KPI_ReachDRR_CPD","KPI_ReachS_CPD","KPI_ReachL_CPD", "KPI_ReachH_CPD","KPI_ReachWASH_CPD","KPI_ReachM_CPD","KPI_ReachCTP_CPD","KPI_ReachSI_CPD","KPI_GB_Tot","KPI_pr_sex","KPI_sg_sex","KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_noLocalUnits","KPI_IncomeLC_CHF","KPI_expenditureLC_CHF"]
kpi_float=kpi_code.copy()
kpi_float.remove("KPI_pr_sex")
kpi_float.remove("KPI_sg_sex")
time_series=api_function(years,kpi_code,kpi_float,api_key)
time_series

2016
2017
2018
2019
2020
number of KPI downloaded: 40


,KPI ID,KPI_DON_code,NSO_DON_name,NSO_ZON_name,iso_3,KPI_Year,KPI_DonBlood_Tot,KPI_GB_Tot,KPI_IncomeLC_CHF,KPI_PStaff_Tot,...,KPI_ReachS_D_Tot,KPI_ReachS_I,KPI_ReachWASH_CPD,KPI_ReachWASH_D_Tot,KPI_ReachWASH_I,KPI_TrainFA_Tot,KPI_expenditureLC_CHF,KPI_noLocalUnits,KPI_pr_sex,KPI_sg_sex
0,DAF0012016,DAF001,Afghan Red Crescent,Asia Pacific,AFG,2016,178.0,NaN,1.994429e+07,2247.0,...,0.0,NaN,0.0,0.0,NaN,4020.0,2.054210e+07,58.0,NaN,NaN
1,DAL0012016,DAL001,Albanian Red Cross,Europe and Central Asia,ALB,2016,6181.0,NaN,1.348774e+06,54.0,...,0.0,NaN,0.0,0.0,NaN,53259.0,1.087271e+06,39.0,NaN,NaN
2,DDZ0012016,DDZ001,Algerian Red Crescent,Middle East and North Africa,DZA,2016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN
3,DUS0012016,DUS001,American Red Cross,Americas,USA,2016,2720640.0,NaN,2.604914e+09,20817.0,...,56850.0,0.0,0.0,0.0,NaN,2302312.0,2.665040e+09,264.0,NaN,NaN
4,DAD0012016,DAD001,Andorran Red Cross,Europe and Central Asia,AND,2016,1028.0,NaN,2.029149e+06,23.0,...,0.0,NaN,101.0,101.0,0.0,2257.0,9.511040e+05,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,DVE0012020,DVE001,Venezuelan Red Cross,Americas,VEN,2020,428.0,11.0,0.000000e+00,742.0,...,13.0,NaN,302188.0,302188.0,NaN,2984.0,0.000000e+00,42.0,Male,Male
956,DVN0012020,DVN001,Vietnam Red Cross Society,Asia Pacific,VNM,2020,1405336.0,5.0,5.918624e+06,16806.0,...,9485.0,NaN,127406.0,127406.0,NaN,62961.0,2.976457e+06,17127.0,Female,Male
957,DYE0012020,DYE001,Yemen Red Crescent Society,Middle East and North Africa,YEM,2020,NaN,6.0,4.452352e+06,476.0,...,598808.0,NaN,415482.0,415482.0,NaN,2495.0,3.709803e+06,31.0,Male,Male
958,DZM0012020,DZM001,Zambia Red Cross Society,Africa,ZMB,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2. Run imputing missing values

In [4]:
from imp_function import imputing_mean,imputing_na

In [5]:
# drop from 0 to 2011
data_clean=time_series.query('KPI_Year >= 2012 & KPI_Year < 2021')
# filter out columns that can be ignored: 64 columns dropped (see columns 1134 to 1070)
# The only reporting year FDRS didn't count with 100% reporting was 2019. Therefore, lets divide the dataset in 2: 
# 2012-2017, 2018-2020. This facilitates the code as KPI_WasSubmitted variable (which is being used to filter the data) 
# was implemented in 2017 and all years before have NaN 
# 1st subset:
data_clean1=data_clean.query('KPI_Year >= 2012 & KPI_Year < 2018').copy()
# 2nd subset:
data_clean2=data_clean.query('KPI_Year >= 2018').copy()
data_clean2_na_columns=data_clean2.copy()
#Imputing:People reached sections 
selected_cols=["KPI_DonBlood_Tot","KPI_TrainFA_Tot","KPI_ReachDRER_D_Tot","KPI_ReachDRER_I","KPI_ReachDRR_D_Tot","KPI_ReachDRR_I","KPI_ReachLTSPD_D_Tot","KPI_ReachLTSPD_I","KPI_ReachS_D_Tot","KPI_ReachS_I","KPI_ReachL_D_Tot","KPI_ReachL_I","KPI_ReachM_D_Tot","KPI_ReachM_I","KPI_ReachCTP_D_Tot","KPI_ReachCTP_I","KPI_ReachSI_D_Tot","KPI_ReachSI_I","KPI_ReachH_D_Tot","KPI_ReachH_I","KPI_ReachWASH_I","KPI_ReachWASH_D_Tot","KPI_ReachDRER_CPD","KPI_ReachLTSPD_CPD","KPI_ReachDRR_CPD","KPI_ReachS_CPD","KPI_ReachL_CPD", "KPI_ReachH_CPD","KPI_ReachWASH_CPD","KPI_ReachM_CPD","KPI_ReachCTP_CPD","KPI_ReachSI_CPD"]
imputing_mean(data_clean2_na_columns,selected_cols,data_clean2)
# Imputing : NS Finance and Parnership & NS Governance and Structure Section
selected_cols_2d=["KPI_GB_Tot","KPI_pr_sex","KPI_sg_sex","KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_noLocalUnits","KPI_IncomeLC_CHF" ,"KPI_expenditureLC_CHF"]
# 18.11.21Comment : After reviewing with the Team, "supported1" and "received_support1" have been imputed seperatly in the script FDRS Network Data Transformation
imputing_na(data_clean2_na_columns,selected_cols_2d)
# concatenate the two sub_datasets
fdrs_na_columns = [data_clean1, data_clean2_na_columns]
fdrs_data_fdrs_na_columns = pd.concat(fdrs_na_columns)


KPI_DonBlood_Tot
KPI_TrainFA_Tot
KPI_ReachDRER_D_Tot
KPI_ReachDRER_I
KPI_ReachDRR_D_Tot
KPI_ReachDRR_I
KPI_ReachLTSPD_D_Tot
KPI_ReachLTSPD_I
KPI_ReachS_D_Tot
KPI_ReachS_I
KPI_ReachL_D_Tot
KPI_ReachL_I
KPI_ReachM_D_Tot
KPI_ReachM_I
KPI_ReachCTP_D_Tot
KPI_ReachCTP_I
KPI_ReachSI_D_Tot
KPI_ReachSI_I
KPI_ReachH_D_Tot
KPI_ReachH_I
KPI_ReachWASH_I
KPI_ReachWASH_D_Tot
KPI_ReachDRER_CPD
KPI_ReachLTSPD_CPD
KPI_ReachDRR_CPD
KPI_ReachS_CPD
KPI_ReachL_CPD
KPI_ReachH_CPD
KPI_ReachWASH_CPD
KPI_ReachM_CPD
KPI_ReachCTP_CPD
KPI_ReachSI_CPD
KPI_GB_Tot
KPI_pr_sex
KPI_sg_sex
KPI_PeopleVol_Tot
KPI_PStaff_Tot
KPI_noLocalUnits
KPI_IncomeLC_CHF
KPI_expenditureLC_CHF


In [6]:
fdrs_data_fdrs_na_columns[fdrs_data_fdrs_na_columns["KPI_Year"]==2020][["NSO_DON_name","KPI_Year","KPI_PeopleVol_Tot"]].sum()

NSO_DON_name         Afghan Red CrescentAlbanian Red CrossAlgerian ...
KPI_Year                                                        387840
KPI_PeopleVol_Tot                                           14916410.0
dtype: object

### 3. Get imputed values t-1 from backoffice to compare with imputed values t

In [7]:
kpi_code=["KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_DonBlood_Tot","KPI_TrainFA_Tot","KPI_noLocalUnits","KPI_ReachDRER_CPD","KPI_ReachLTSPD_CPD","KPI_ReachDRR_CPD","KPI_ReachS_CPD","KPI_ReachL_CPD","KPI_ReachH_CPD","KPI_ReachWASH_CPD","KPI_ReachM_CPD","KPI_ReachCTP_CPD","KPI_ReachSI_CPD","KPI_IncomeLC_CHF","KPI_expenditureLC_CHF"]
kpi_code= [s + "_IP" for s in kpi_code]
kpi_float=kpi_code.copy()


Get imputed values from backoffice

In [8]:
time_series_imputed=api_function_imputed(years,kpi_code,kpi_float,api_key)
time_series_imputed

2016
2017
2018
2019
2020
number of KPI downloaded: 17


,KPI ID,KPI_DON_code,NSO_DON_name,NSO_ZON_name,iso_3,KPI_Year,nan,KPI_DonBlood_Tot_IP,KPI_IncomeLC_CHF_IP,KPI_PStaff_Tot_IP,...,KPI_ReachH_CPD_IP,KPI_ReachLTSPD_CPD_IP,KPI_ReachL_CPD_IP,KPI_ReachM_CPD_IP,KPI_ReachSI_CPD_IP,KPI_ReachS_CPD_IP,KPI_ReachWASH_CPD_IP,KPI_TrainFA_Tot_IP,KPI_expenditureLC_CHF_IP,KPI_noLocalUnits_IP
0,DAF0012016,DAF001,Afghan Red Crescent,Asia Pacific,AFG,2016,NaN,178.0,1.994429e+07,2247.0,...,0.0,0.0,10892.0,0.0,3157384.0,0.0,0.0,4020.0,2.054210e+07,58.0
1,DAL0012016,DAL001,Albanian Red Cross,Europe and Central Asia,ALB,2016,NaN,6181.0,1.348774e+06,54.0,...,132770.0,48235.0,0.0,260.0,2541.0,0.0,0.0,53259.0,1.087271e+06,39.0
2,DDZ0012016,DDZ001,Algerian Red Crescent,Middle East and North Africa,DZA,2016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0
3,DUS0012016,DUS001,American Red Cross,Americas,USA,2016,NaN,2720640.0,2.604914e+09,20817.0,...,3945270.0,4288126.0,82231.0,0.0,0.0,56850.0,0.0,2302312.0,2.665040e+09,264.0
4,DAD0012016,DAD001,Andorran Red Cross,Europe and Central Asia,AND,2016,NaN,1028.0,2.029149e+06,23.0,...,1903.0,1903.0,0.0,11.0,221.0,0.0,101.0,2257.0,9.511040e+05,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,DVE0012020,DVE001,Venezuelan Red Cross,Americas,VEN,2020,NaN,428.0,0.000000e+00,742.0,...,578073.0,694959.0,7954.0,35387.0,38026.0,13.0,302188.0,2984.0,0.000000e+00,42.0
956,DVN0012020,DVN001,Vietnam Red Cross Society,Asia Pacific,VNM,2020,NaN,1405336.0,5.918624e+06,16806.0,...,3939634.0,3960018.0,21353.0,0.0,0.0,9485.0,127406.0,62961.0,2.976457e+06,17127.0
957,DYE0012020,DYE001,Yemen Red Crescent Society,Middle East and North Africa,YEM,2020,NaN,0.0,4.452352e+06,476.0,...,3671527.0,1699422.0,635532.0,403627.0,154255.0,598808.0,415482.0,2495.0,3.709803e+06,31.0
958,DZM0012020,DZM001,Zambia Red Cross Society,Africa,ZMB,2020,NaN,0.0,7.896810e+05,61.0,...,39616.0,0.0,0.0,14073.0,0.0,10757.0,130407.0,2735.0,1.171450e+06,45.0


Code the values that are different.   
0 for values that do not need to be updated,   
1 for values that need to be updated. 

In [9]:
kpi_code=["KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_DonBlood_Tot","KPI_TrainFA_Tot","KPI_noLocalUnits","KPI_ReachDRER_CPD","KPI_ReachLTSPD_CPD","KPI_ReachDRR_CPD","KPI_ReachS_CPD","KPI_ReachL_CPD","KPI_ReachH_CPD","KPI_ReachWASH_CPD","KPI_ReachM_CPD","KPI_ReachCTP_CPD","KPI_ReachSI_CPD","KPI_IncomeLC_CHF","KPI_expenditureLC_CHF"]
# Convert to type int to remove decimals from calculations
#fdrs_data_fdrs_na_columns[kpi_code]=fdrs_data_fdrs_na_columns[kpi_code].fillna(0).astype('int')
#fdrs_data_fdrs_na_columns[kpi_code]=fdrs_data_fdrs_na_columns[kpi_code].replace(0,np.nan)



In [10]:
kpi_code=["KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_DonBlood_Tot","KPI_TrainFA_Tot","KPI_noLocalUnits","KPI_ReachDRER_CPD","KPI_ReachLTSPD_CPD","KPI_ReachDRR_CPD","KPI_ReachS_CPD","KPI_ReachL_CPD","KPI_ReachH_CPD","KPI_ReachWASH_CPD","KPI_ReachM_CPD","KPI_ReachCTP_CPD","KPI_ReachSI_CPD","KPI_IncomeLC_CHF","KPI_expenditureLC_CHF"]
kpi_code_imp= [s + "_IP" for s in kpi_code]
kpi_code_binary = [s + "_binary" for s in kpi_code]


for kpi in list(kpi_code):
    kpi_code_imp=kpi+"_IP"
    kpi_code_binary=kpi+"_binary"
    fdrs_data_fdrs_na_columns.loc[time_series_imputed[kpi_code_imp] == fdrs_data_fdrs_na_columns[kpi],kpi_code_binary]=0 #0 if same
    fdrs_data_fdrs_na_columns.loc[time_series_imputed[kpi_code_imp] != fdrs_data_fdrs_na_columns[kpi],kpi_code_binary]=1 #1 if different
    
    fdrs_data_fdrs_na_columns.loc[fdrs_data_fdrs_na_columns[kpi].isna(),kpi_code_binary]=2
    #To be uncommented when the data is ready
    fdrs_data_fdrs_na_columns.loc[fdrs_data_fdrs_na_columns[kpi_code_binary]==2,kpi_code_binary]=0

            


In [11]:
print(fdrs_data_fdrs_na_columns[kpi_code].dtypes)

KPI_PeopleVol_Tot        float64
KPI_PStaff_Tot           float64
KPI_DonBlood_Tot         float64
KPI_TrainFA_Tot          float64
KPI_noLocalUnits         float64
KPI_ReachDRER_CPD        float64
KPI_ReachLTSPD_CPD       float64
KPI_ReachDRR_CPD         float64
KPI_ReachS_CPD           float64
KPI_ReachL_CPD           float64
KPI_ReachH_CPD           float64
KPI_ReachWASH_CPD        float64
KPI_ReachM_CPD           float64
KPI_ReachCTP_CPD         float64
KPI_ReachSI_CPD          float64
KPI_IncomeLC_CHF         float64
KPI_expenditureLC_CHF    float64
dtype: object


### 4. Post method - interact with Imp_Var

Years to impute:   

In [12]:
years=[2016,2017,2018,2019,2020]

Variables to impute :   
total in front end

In [15]:
f = open("api_key.txt", "r")
api_key=f.readline()

kpi_code=["KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_DonBlood_Tot","KPI_TrainFA_Tot","KPI_noLocalUnits","KPI_ReachDRER_CPD","KPI_ReachLTSPD_CPD","KPI_ReachDRR_CPD","KPI_ReachS_CPD","KPI_ReachL_CPD","KPI_ReachH_CPD","KPI_ReachWASH_CPD","KPI_ReachM_CPD","KPI_ReachCTP_CPD","KPI_ReachSI_CPD","KPI_IncomeLC_CHF","KPI_expenditureLC_CHF"]
payload={}
files={}
headers = {}

In [16]:
#fdrs_data_fdrs_na_columns_test.fillna(0,inplace=True)
for kpi in list(kpi_code):
    kpi_binary = kpi + "_binary"
    for year in years:
        for kpi_don_code in list(set(fdrs_data_fdrs_na_columns["KPI_DON_code"])): 
            for bi in (fdrs_data_fdrs_na_columns[kpi_binary][(fdrs_data_fdrs_na_columns["KPI_Year"]==year)&(fdrs_data_fdrs_na_columns["KPI_DON_code"]==kpi_don_code)]):
                binary = bi
                if binary == 1.0: 
                    for label, content in (fdrs_data_fdrs_na_columns[kpi][(fdrs_data_fdrs_na_columns["KPI_Year"]==year)&(fdrs_data_fdrs_na_columns["KPI_DON_code"]==kpi_don_code)]).items():
                        value = content
                        if type(value)==float:
                            if not np.isnan(value):
                                value=np.int64(value)
                                value = value
                        #if value == "nan":
                        if np.isnan(value):
                            value = ""
                        kpi_post=kpi+"_IP"
                        url = (f"https://data-api.ifrc.org/api/ImputedKPI?apiKey={api_key}&kpicode={kpi_post}&year={year}&don_code={kpi_don_code}&value={value}&user=simon.weiss@ifrc.org")
                        r = requests.post(url,headers=headers, data=payload, files=files)
                        #print(url,r.text)
                        #print(r.request.body)
                        #print(r.request.headers)

                        

send by hand timeout data

In [39]:
df1 = pd.DataFrame([datetime.now()], columns=['Lattest Update Date'])
df1.to_excel("lattest_update.xlsx", index=False)


Time to execute all : before  optimization : 111min   
After optimization : 2min